# Music name generator
Well here we are. I'm going to find THE best music name, thanks to machine learning

In [1]:
import numpy as np 
import pandas as pd 
import string
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/home/goznalo/Programming/Python/musicnames'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


%matplotlib inline

/home/goznalo/Programming/Python/musicnames/artists.csv
/home/goznalo/Programming/Python/musicnames/miusic.ipynb
/home/goznalo/Programming/Python/musicnames/.git/HEAD
/home/goznalo/Programming/Python/musicnames/.git/config
/home/goznalo/Programming/Python/musicnames/.git/COMMIT_EDITMSG
/home/goznalo/Programming/Python/musicnames/.git/description
/home/goznalo/Programming/Python/musicnames/.git/index
/home/goznalo/Programming/Python/musicnames/.git/info/exclude
/home/goznalo/Programming/Python/musicnames/.git/hooks/pre-push.sample
/home/goznalo/Programming/Python/musicnames/.git/hooks/pre-applypatch.sample
/home/goznalo/Programming/Python/musicnames/.git/hooks/pre-rebase.sample
/home/goznalo/Programming/Python/musicnames/.git/hooks/commit-msg.sample
/home/goznalo/Programming/Python/musicnames/.git/hooks/fsmonitor-watchman.sample
/home/goznalo/Programming/Python/musicnames/.git/hooks/applypatch-msg.sample
/home/goznalo/Programming/Python/musicnames/.git/hooks/post-update.sample
/home/goz

### Importing the data
Let's import the data from the kaggle dataset music-artists-popularity, using the Pandas library.

In [2]:
filename = '/home/goznalo/Programming/Python/musicnames/artists.csv'
dataset = pd.read_csv(filename, usecols = [2]) # Obtain the artist name column.

/usr/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
dataset.head # preview of the data

<bound method NDFrame.head of                  artist_lastfm
0                     Coldplay
1                    Radiohead
2        Red Hot Chili Peppers
3                      Rihanna
4                       Eminem
...                        ...
1466078                    NaN
1466079                    NaN
1466080                    NaN
1466081                    NaN
1466082                    NaN

[1466083 rows x 1 columns]>

### Clearing, part 1
We don't want any of the NaN entries, nor duplicate names. We will also convert it to a numpy array for posterior transformations.

In [4]:
df = dataset.dropna(axis = 0,how = 'any',thresh = None).drop_duplicates(subset=None) #removing NaN's and converting to numpy.
names = np.squeeze(np.asarray(df))

In [5]:
numnames = names.shape[0]

In [6]:
names[0]

'Coldplay'

### Clearing, part 2
We will now remove all those names using non-standard characters. What I mean by non-standard is all those characters not being latin ones, nor punctuation ones, nor digits, nor spaces. For instance, we will get rid of those using chinese characters or greek letters.

We quickly run intro trouble, as the .isalnum() method applied to these non-latin characters returns True. We need to come up with a solution. One way around this is by using the string module. With it, the string.printable characters are those we want to allow in our names, therefore we make a set out of them, which will be used against the names of the database in a loop.

In [7]:
validchars = {}
validchars[0] = set(string.printable)
print(validchars[0])

{'4', '%', 'i', '9', '*', 'a', '~', '!', 'j', '"', 'K', 'F', 'X', 'q', '/', 'c', '|', '6', '\r', "'", 'P', 'O', '0', 'I', 'm', 'y', ')', 'W', '\x0c', 'A', 'd', 'w', 'B', '`', '1', 'L', '+', '?', '2', 'b', ' ', 'M', ':', '-', 'v', 'l', 'U', 'g', '@', '3', '_', '=', '>', 'e', 'p', 'R', '$', 'N', 'f', '\x0b', '5', '{', ']', '[', '\n', '&', '<', '\t', 'Q', 'D', 'H', 'z', 'Y', 'u', 'x', 'Z', '^', '.', 'V', 'r', '\\', 'h', 'J', 'G', 'E', '7', '8', 's', ';', 't', 'S', '}', ',', 'o', '#', 'k', 'T', '(', 'n', 'C'}


Now comes the loop. An initial approach was creating a new object, a numpy array, to which we would append each valid name, therefore discarding the rest of the names with invalid characters. However, the appending operation makes the algorithm take exponential time of completion. Instead, we can save the index of each invalid word in a list, which we then feed to the np.delete() function to remove those entries of the "names" array

In [8]:
deletelist = {}
def deletechars(validcharacters, listofnames):  # We define it as a function, as we will need it in the next steps
    #clearlist = np.zeros((1,1))
    i = 0 # counts each iteration, i.e. the 
    j = 0 # counts each invalid word, used to check every now and then that names are correctly discarded.
    deletelist = []
    for name in listofnames:
        if not all(char in validcharacters for char in name):
            deletelist.append(i)
            if j%10000 == 0:
                print("invalid: " + name)
            j += 1
        #else:
        #    clearlist = np.append(clearlist, name[0])
        i += 1
        if i%50000 == 0:
            print(str(i) + " cases inspected.")
    return deletelist

deletelist[0] = deletechars(validchars[0], names)

names[deletelist[0][0]] # check that this corresponds to the first invalid word, Beyoncé

invalid: Beyoncé
50000 cases inspected.
100000 cases inspected.
150000 cases inspected.
invalid: Min Ronnersjö
200000 cases inspected.
250000 cases inspected.
300000 cases inspected.
invalid: Kaszás Attila
350000 cases inspected.
400000 cases inspected.
invalid: E=MC²
450000 cases inspected.
500000 cases inspected.
invalid: Natércia Maria
550000 cases inspected.
600000 cases inspected.
invalid: 堀米ゆず子
650000 cases inspected.
700000 cases inspected.
invalid: Fütumche!
750000 cases inspected.
invalid: Léo Clarens
800000 cases inspected.
850000 cases inspected.
invalid: 권기욱
900000 cases inspected.
invalid: Arbeiter-Chor (Irmler-Männerchor)
950000 cases inspected.


'Beyoncé'

In [9]:
clearedlist = {}
clearedlist[0] = np.delete(names, deletelist[0])
clearedlist[0][0:22] # Check that Beyoncé has been correctly removed.

array(['Coldplay', 'Radiohead', 'Red Hot Chili Peppers', 'Rihanna',
       'Eminem', 'The Killers', 'Kanye West', 'Nirvana', 'Muse', 'Queen',
       'Foo Fighters', 'Linkin Park', 'Lady Gaga', 'The Rolling Stones',
       'Daft Punk', 'Green Day', 'Katy Perry', 'The Beatles', 'Oasis',
       'Gorillaz', 'Michael Jackson', 'Maroon 5'], dtype=object)

In [10]:
np.savetxt("clearedlist2.csv", clearedlist[0], delimiter=",", fmt='%s') #save the list to a csv file.

### Clearing, part 3: choose your own adventure
We will now distinguish four different cases which can be studied, with decreasing complexity. 
1. The full list, as is.
2. The list, having removed punctuation signs.
3. The list, having removed both punctuation characters and names with more than two words.
4. The list, having removed both punctuation characters and names with more than one word.